## Machine Learning Model

Recently, driven by the need to maximise the use of big data, ML methods has been widely employed in many application across various indutries. However, ML is deemed to be increadibily difficult to interpret due to its nature model complexity. This project focuses on explaining why model makes certain predictions. 

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics, linear_model, tree, naive_bayes, neighbors, ensemble,\
                    neural_network, svm, decomposition, manifold
from rulefit import RuleFit
import statsmodels.api as sm
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from interpret.perf import ROC
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer

## Data Preprocessing

In [2]:
house =  pd.read_csv("src/train.csv")
ml_dt = house.drop(["PoolQC", "Fence", "FireplaceQu", "Alley", "MiscFeature", "Id"], axis=1)
missing = ml_dt.isnull().sum().sort_values(ascending=False)
missing = missing[missing != 0]
missing

LotFrontage     259
GarageType       81
GarageYrBlt      81
GarageFinish     81
GarageQual       81
GarageCond       81
BsmtFinType2     38
BsmtExposure     38
BsmtQual         37
BsmtCond         37
BsmtFinType1     37
MasVnrArea        8
MasVnrType        8
Electrical        1
dtype: int64

In [3]:
ml_dt = ml_dt.dropna()
ml_dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1094 non-null   int64  
 1   MSZoning       1094 non-null   object 
 2   LotFrontage    1094 non-null   float64
 3   LotArea        1094 non-null   int64  
 4   Street         1094 non-null   object 
 5   LotShape       1094 non-null   object 
 6   LandContour    1094 non-null   object 
 7   Utilities      1094 non-null   object 
 8   LotConfig      1094 non-null   object 
 9   LandSlope      1094 non-null   object 
 10  Neighborhood   1094 non-null   object 
 11  Condition1     1094 non-null   object 
 12  Condition2     1094 non-null   object 
 13  BldgType       1094 non-null   object 
 14  HouseStyle     1094 non-null   object 
 15  OverallQual    1094 non-null   int64  
 16  OverallCond    1094 non-null   int64  
 17  YearBuilt      1094 non-null   int64  
 18  YearRemo

Remove missing values technique is not an idea solution for model performance in general but this project focuses more on  model intrepretaion rather than building the correct model. 25% of the data have been ommitted which is consider a huge portion.

In [4]:
object_features = ml_dt.select_dtypes(include=['object'])
numerical_features = ml_dt.select_dtypes(include=['int64', "float64"])

object_features_encode = pd.get_dummies(object_features)
numerical_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1459
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1094 non-null   int64  
 1   LotFrontage    1094 non-null   float64
 2   LotArea        1094 non-null   int64  
 3   OverallQual    1094 non-null   int64  
 4   OverallCond    1094 non-null   int64  
 5   YearBuilt      1094 non-null   int64  
 6   YearRemodAdd   1094 non-null   int64  
 7   MasVnrArea     1094 non-null   float64
 8   BsmtFinSF1     1094 non-null   int64  
 9   BsmtFinSF2     1094 non-null   int64  
 10  BsmtUnfSF      1094 non-null   int64  
 11  TotalBsmtSF    1094 non-null   int64  
 12  1stFlrSF       1094 non-null   int64  
 13  2ndFlrSF       1094 non-null   int64  
 14  LowQualFinSF   1094 non-null   int64  
 15  GrLivArea      1094 non-null   int64  
 16  BsmtFullBath   1094 non-null   int64  
 17  BsmtHalfBath   1094 non-null   int64  
 18  FullBath

In [5]:
corr = numerical_features.corr()
abs(corr["SalePrice"]).sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.795437
GrLivArea        0.707481
GarageCars       0.652103
GarageArea       0.620772
TotalBsmtSF      0.617741
1stFlrSF         0.617692
FullBath         0.578299
TotRmsAbvGrd     0.560521
YearBuilt        0.523434
YearRemodAdd     0.519806
GarageYrBlt      0.502248
MasVnrArea       0.485409
Fireplaces       0.458182
BsmtFinSF1       0.378678
LotFrontage      0.343978
OpenPorchSF      0.338600
WoodDeckSF       0.330286
2ndFlrSF         0.302569
LotArea          0.302268
HalfBath         0.259469
BsmtFullBath     0.223948
BsmtUnfSF        0.191247
BedroomAbvGr     0.168489
EnclosedPorch    0.161711
OverallCond      0.138511
KitchenAbvGr     0.115382
ScreenPorch      0.106479
PoolArea         0.092085
MSSubClass       0.089478
MoSold           0.052584
BsmtHalfBath     0.041341
BsmtFinSF2       0.036923
MiscVal          0.036001
3SsnPorch        0.033947
YrSold           0.006723
LowQualFinSF     0.003541
Name: SalePrice, dtype: float64

In [6]:
rand = 10
x = numerical_features.drop(["SalePrice"], axis=1).copy()
y = numerical_features["SalePrice"]
X_train, X_test, y_train_reg, y_test_reg = train_test_split(x, y, test_size=0.15, random_state=rand)

ct = ColumnTransformer([('normalize', Normalizer(), x.columns)], remainder='passthrough')
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)
y_train_reg = np.log(y_train_reg )
y_test_reg = np.log(y_test_reg)

## Training and Evaluating various Regression Models

1. instance-based machine learning model, *lazy learner*
    - during inference, it employs training data to calculate the similarity between points and generate predictions based on that.


2. model-based machine learning technique, *eager learner*
    - during inference, it learns formula, coefficient, bias, paramters, weight which it then leverages to estimate the true value.

we compare and contrast a few linear models.

    1. Linear Regression
        - as the name suggest, the model assumes linearity between relationhips of the features and the target variable. Prediction is a result of a linear combination of the X features.
    
    2. Polynomial Regression
        - extend linear model by adding polynomial terms.
    
    3. Linear interact 
        - capture the interactions between the features. 
    
    4. Ridge Regression
        - Although OLS (ordinary least square) does a good job in fitting the model to the features by minimising the error term, it does it with without considering overfitting. OLS treats each feature equally, so the model becomes more complex as each variable is added. Ridge regression uses regularization to minimise the weight of coefficients that do not contribute to the outcome with penalty term called L2 norm. 
    
    5. RuleFit 
        - is a regularised linear regression expanded to include feature interactions in the forms of rules. it can be used to solve both classification and regression problems. 
    
    6. knn (K-nearest neighbour algorithms)
        - method based on locality assumptions. it predicts a values correspond to similiar data points that are close to each other.
    
    7. Random forest
        - combine many decision tress together to make decision. different trees randomly sample feature combination on random sample data. ensemble methods mean that the model combine more than one submodels to generate predicitons. 

In [7]:

reg_models = {
        #Generalized Linear Models (GLMs)
        'linear':{'model': linear_model.LinearRegression()}, 
        'linear_poly':{'model': make_pipeline(PolynomialFeatures(degree=2),
                              linear_model.LinearRegression(fit_intercept=False)) },
        'linear_interact':{'model': make_pipeline(PolynomialFeatures(interaction_only=True),
                              linear_model.LinearRegression(fit_intercept=False)) },
        'ridge':{'model': linear_model.RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]) }, 
        #Trees  
        'decision_tree':{'model': tree.DecisionTreeRegressor(max_depth=7, random_state=rand)},
        #Ensemble Methods
        'random_forest':{'model':ensemble.RandomForestRegressor(max_depth=7, random_state=rand)}, 
        #RuleFit
        'rulefit':{'model': RuleFit(max_rules=150, rfmode='regress', random_state=rand)}, 
        #Nearest Neighbors
        'knn':{'model': neighbors.KNeighborsRegressor(n_neighbors=7)}, 
        #Neural Networks
        'mlp':{'model':neural_network.MLPRegressor(hidden_layer_sizes=(21,), max_iter=500, 
                                                   early_stopping=True, random_state=rand)}}

In [8]:
for model_name in reg_models.keys():
    if model_name != 'rulefit':
        fitted_model = reg_models[model_name]['model'].fit(X_train, y_train_reg)
    else:
        fitted_model = reg_models[model_name]['model'].fit(X_train, y_train_reg.values, x.columns)
    y_train_pred = fitted_model.predict(X_train)
    y_test_pred = fitted_model.predict(X_test)
    reg_models[model_name]['fitted'] = fitted_model
    reg_models[model_name]['preds'] = y_test_pred
    reg_models[model_name]['RMSE_train'] = math.sqrt(metrics.mean_squared_error(y_train_reg, y_train_pred))
    reg_models[model_name]['RMSE_test'] = math.sqrt(metrics.mean_squared_error(y_test_reg, y_test_pred))
    reg_models[model_name]['R2_test'] = metrics.r2_score(y_test_reg, y_test_pred)

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.10170921732962768, tolerance: 0.009595433802159827
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1746540301319861, tolerance: 0.009595433802159827
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2233241630942544, tolerance: 0.009595433802159827
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not con

In [9]:
reg_metrics = pd.DataFrame.from_dict(reg_models, 'index')[['RMSE_train', 'RMSE_test', 'R2_test']]
reg_metrics.sort_values(by='RMSE_test').style.\
    background_gradient(cmap='viridis', low=1, high=0.3, subset=['RMSE_train', 'RMSE_test']).\
    background_gradient(cmap='plasma', low=0.3, high=1, subset=['R2_test'])

,RMSE_train,RMSE_test,R2_test
rulefit,0.134430,0.149997,0.860710
linear,0.157760,0.164305,0.832869
random_forest,0.114592,0.196315,0.761405
ridge,0.185352,0.198226,0.756737
knn,0.187467,0.241010,0.640395
decision_tree,0.136630,0.259737,0.582340
mlp,0.449649,0.484771,-0.454883
linear_interact,0.076284,5.227724,-168.192022
linear_poly,0.072610,392661.470059,-954533590673.008179


When ML methods are employed, we are often dealing with complex relationships between features. 
One way of reducing the complexity in visualising is by doing dimensionality reduction methods. 

Dimensionality Reduction technique has many application throughout any project pipilines. some leverage the technique purely to find pattern visually, some extended the concept to feature selection and enginering, oulier detection, model debugging.  


## Feature Importance

Understanding intrinsically interpretable (white-box) model 

### Linear Regression

In [10]:
coefs_lm = reg_models['linear']['fitted'].coef_
intercept_lm = reg_models['linear']['fitted'].intercept_
print('coefficients:\t%s' % coefs_lm)
print('intercept:\t%s' % intercept_lm)

coefficients:	[-6.19489970e+00  1.18804847e+00 -1.15407236e+00  7.25507242e+02
  4.79675247e+02  2.01683602e+01  1.13721474e+01  9.28755261e-01
  1.24026274e+00  4.27400783e-01  1.08695436e-01  1.77635896e+00
 -3.37727322e-01  1.14487182e+00  9.00681671e-01  1.70782617e+00
  9.28267386e+01 -3.94785723e+02  5.15376950e+01  1.67528544e+02
 -3.04930400e+02 -1.26683921e+03  7.62684563e+01  2.98179659e+02
  4.28968681e+00 -9.46873067e+01  1.81707290e+00  4.13861433e-01
  2.81382092e+00  1.30317865e+00  1.45189270e+00  1.78700910e+00
  1.32729709e+00 -7.34156994e-01 -1.57921895e+01 -4.32487088e+01]
intercept:	13.520823415750591


In [11]:
print('ŷ = %0.2f + %0.4fX₁ + %0.4fX₂ + %0.3fX₃ + ...' %\
      (intercept_lm, coefs_lm[0], coefs_lm[1], coefs_lm[2]))

ŷ = 13.52 + -6.1949X₁ + 1.1880X₂ + -1.154X₃ + ...


In [12]:
coef_df = pd.DataFrame({'feature':x.columns.values.tolist(),\
                        'coef': coefs_lm})
print(coef_df)

          feature         coef
0      MSSubClass    -6.194900
1     LotFrontage     1.188048
2         LotArea    -1.154072
3     OverallQual   725.507242
4     OverallCond   479.675247
5       YearBuilt    20.168360
6    YearRemodAdd    11.372147
7      MasVnrArea     0.928755
8      BsmtFinSF1     1.240263
9      BsmtFinSF2     0.427401
10      BsmtUnfSF     0.108695
11    TotalBsmtSF     1.776359
12       1stFlrSF    -0.337727
13       2ndFlrSF     1.144872
14   LowQualFinSF     0.900682
15      GrLivArea     1.707826
16   BsmtFullBath    92.826739
17   BsmtHalfBath  -394.785723
18       FullBath    51.537695
19       HalfBath   167.528544
20   BedroomAbvGr  -304.930400
21   KitchenAbvGr -1266.839205
22   TotRmsAbvGrd    76.268456
23     Fireplaces   298.179659
24    GarageYrBlt     4.289687
25     GarageCars   -94.687307
26     GarageArea     1.817073
27     WoodDeckSF     0.413861
28    OpenPorchSF     2.813821
29  EnclosedPorch     1.303179
30      3SsnPorch     1.451893
31    Sc

In [13]:
x_train_df = pd.DataFrame(X_train)

#important data manipulation
def transform_col_name(y_list, x_list):
    #take a list of original colnames and convert it to a new colname
    colum_x = {}
    for i, v in zip(y_list, x_list):
        colum_x[i] =v 
    return colum_x

colum_x_name = transform_col_name(x_train_df.columns, numerical_features.columns)
x_train_df.rename(columns=colum_x_name, inplace=True)
y_train_df = pd.DataFrame({"SalesPrice": y_train_reg})
y_index= y_train_df.reset_index().drop(columns = ["index"]) #indexes of both df must be identical so that y and x train can be plugged into stast regression model

**Note that** Extracting more information about regression model can't be done through sklearn. We need to use stastmodel to get this information. 

In [14]:
lineareg = sm.OLS(y_index, sm.add_constant(x_train_df))
lineareg = lineareg.fit()
lineareg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             SalesPrice   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     137.1
Date:                Sun, 13 Mar 2022   Prob (F-statistic):               0.00
Time:                        17:58:07   Log-Likelihood:                 397.37
No. Observations:                 929   AIC:                            -724.7
Df Residuals:                     894   BIC:                            -555.5
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            13.5208      0.243     55.533      0.000      13.043      13.999
MSSubClass       -6.1949      1.997     -3.102      0.002     -10.114      -2.276
LotFrontage       1.1880      4.246      0.280      0.780      -7.146       9.522
LotArea          -1.1541      0.215     -5.363      0.000      -1.576      -0.732
OverallQual     725.5072     67.070     10.817      0.000     593.874     857.140
OverallCond     479.6752     59.646      8.042      0.000     362.613     596.737
YearBuilt        20.1684      4.123      4.892      0.000      12.077      28.259
YearRemodAdd     11.3721      3.853      2.951      0.003       3.810      18.934
MasVnrArea        0.9288      0.316      2.939      0.003       0.308       1.549
BsmtFinSF1        1.2403      0.180      6.877      0.000       0.886       1.594
BsmtFinSF2        0.4274      0.293      1.458      0.145      -0.148       1.003
BsmtUnfSF         0.1087      0.163      0.669      0.504      -0.210       0.428
TotalBsmtSF       1.7764      0.317      5.605      0.000       1.154       2.398
1stFlrSF         -0.3377      0.441     -0.765      0.444      -1.204       0.528
2ndFlrSF          1.1449      0.371      3.084      0.002       0.416       1.873
LowQualFinSF      0.9007      0.966      0.933      0.351      -0.995       2.796
GrLivArea         1.7078      0.377      4.533      0.000       0.968       2.447
BsmtFullBath     92.8267    140.901      0.659      0.510    -183.708     369.362
BsmtHalfBath   -394.7857    239.372     -1.649      0.099    -864.581      75.010
FullBath         51.5377    165.098      0.312      0.755    -272.487     375.562
HalfBath        167.5285    153.034      1.095      0.274    -132.819     467.876
BedroomAbvGr   -304.9304    101.769     -2.996      0.003    -504.665    -105.195
KitchenAbvGr  -1266.8392    371.168     -3.413      0.001   -1995.301    -538.377
TotRmsAbvGrd     76.2685     70.927      1.075      0.283     -62.934     215.471
Fireplaces      298.1797    103.900      2.870      0.004      94.262     502.097
GarageYrBlt       4.2897      4.169      1.029      0.304      -3.893      12.472
GarageCars      -94.6873    166.086     -0.570      0.569    -420.651     231.276
GarageArea        1.8171      0.607      2.994      0.003       0.626       3.008
WoodDeckSF        0.4139      0.489      0.846      0.398      -0.546       1.374
OpenPorchSF       2.8138      0.963      2.922      0.004       0.924       4.704
EnclosedPorch     1.3032      0.973      1.339      0.181      -0.607       3.213
3SsnPorch         1.4519      2.239      0.648      0.517      -2.943       5.847
ScreenPorch       1.7870      1.116      1.602      0.110      -0.403       3.977
PoolArea          1.3273      1.773      0.749      0.454      -2.153       4.807
MiscVal          -0.7342      0.350     -2.096   

In [15]:
summary_df = lineareg.summary2().tables[1]
summary_df = summary_df.drop(['const']).reset_index().rename(columns={'index':'feature'})
summary_df['t_abs'] = abs(summary_df['t'])
summary_df.sort_values(by='t_abs', ascending=False).style.\
    background_gradient(cmap='plasma_r', low=0, high=0.1, subset=['P>|t|']).\
    background_gradient(cmap='plasma_r', low=0, high=0.1, subset=['t_abs'])

,feature,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],t_abs
3,OverallQual,725.507242,67.069993,10.817166,0.000000,593.874261,857.140223,10.817166
35,YrSold,-43.248709,4.040027,-10.705054,0.000000,-51.177751,-35.319666,10.705054
4,OverallCond,479.675247,59.645751,8.042069,0.000000,362.613239,596.737255,8.042069
8,BsmtFinSF1,1.240263,0.180341,6.877334,0.000000,0.886322,1.594203,6.877334
11,TotalBsmtSF,1.776359,0.316932,5.604850,0.000000,1.154341,2.398377,5.604850
2,LotArea,-1.154072,0.215197,-5.362869,0.000000,-1.576422,-0.731723,5.362869
5,YearBuilt,20.168360,4.122507,4.892257,0.000001,12.077442,28.259278,4.892257
15,GrLivArea,1.707826,0.376791,4.532556,0.000007,0.968328,2.447324,4.532556
21,KitchenAbvGr,-1266.839205,371.167865,-3.413117,0.000671,-1995.301076,-538.377335,3.413117
0,MSSubClass,-6.194900,1.996985,-3.102127,0.001982,-10.114224,-2.275575,3.102127


### Ridge Regression

In [16]:
coefs_ridge = reg_models['ridge']['fitted'].coef_
coef_ridge_df = pd.DataFrame({'feature':x_train_df.columns.values.tolist(),\
                        'coef_linear': coefs_lm,\
                        'coef_ridge': coefs_ridge})
coef_ridge_df.style.\
    background_gradient(cmap='viridis_r', low=0.3, high=0.2, axis=1)

,feature,coef_linear,coef_ridge
0,MSSubClass,-6.194900,-7.185008
1,LotFrontage,1.188048,-0.894458
2,LotArea,-1.154072,-1.372157
3,OverallQual,725.507242,6.948383
4,OverallCond,479.675247,4.133628
5,YearBuilt,20.168360,17.801389
6,YearRemodAdd,11.372147,20.182400
7,MasVnrArea,0.928755,1.238728
8,BsmtFinSF1,1.240263,1.547535
9,BsmtFinSF2,0.427401,0.274104


regularisation can only work if we choose the hyperparameter correctly. in this case, we let sklearn chooses the optimal alpha for us. we can firgure out the optimal aplha by iterating through many possible alpha and firgure out which parameter was the est. The higher the alpha, the higher the regularization. 

## Decision Tree (CART)

There are many kind of decision tree models and most of them are not relly very interpretable because they use ensemble mehods(boosting, bagging and stacking). CART decision tree is one of the few decision trees that is intrepretable because it is build upon a mathematical formula that we can reverse engineer to look for each feature contributions. 

Some features more often in the tree decision.All the sum of the relative decrease in the Gini index throughout the tree is tallied, and the contribution of each feature is a percentage of this reduction:

In [17]:
dt_imp_df = pd.DataFrame({'feature':x_train_df.columns.values.tolist(),\
                        'importance': reg_models['decision_tree']['fitted'].feature_importances_}).\
            sort_values(by='importance', ascending=False)
dt_imp_df

,feature,importance
23,Fireplaces,0.283189
4,OverallCond,0.122410
18,FullBath,0.103960
28,OpenPorchSF,0.072284
26,GarageArea,0.067690
21,KitchenAbvGr,0.040628
35,YrSold,0.037288
24,GarageYrBlt,0.037102
20,BedroomAbvGr,0.028734
8,BsmtFinSF1,0.026329


## Rule Fit

Rule fit is a hybrid model that takes advantage of decision rules and lasso regression. The rulefit implementation used in this project uses gradient boosting decsion tree. 

In [18]:
rulefit_df = reg_models['rulefit']['fitted'].get_rules()
rulefit_df = rulefit_df[rulefit_df.coef != 0].sort_values(by="importance", ascending=False)
rulefit_df

,rule,type,coef,support,importance
35,YrSold,linear,-10.411382,1.000000,0.683979
5,YearBuilt,linear,4.145315,1.000000,0.267909
79,OpenPorchSF <= 0.0029414627933874726 & Firepla...,rule,-0.398059,0.304965,0.183263
82,OpenPorchSF <= 0.0029414627933874726 & Firepla...,rule,0.392782,0.283688,0.177061
67,OpenPorchSF > 0.0014224943588487804 & Fireplac...,rule,0.431023,0.209220,0.175319
...,...,...,...,...,...
51,OpenPorchSF > 0.0015628839610144496 & OverallC...,rule,0.001982,0.223404,0.000825
17,BsmtHalfBath,linear,-38.641732,1.000000,0.000736
98,OpenPorchSF <= 0.0003341914271004498,rule,-0.001343,0.418440,0.000662
121,Fireplaces > 5.5814602092141286e-05 & BsmtFinS...,rule,-0.001653,0.102837,0.000502


The linear coefficient can be interpret as we would any linear regression coeeficient. the rule captures the interactions and can be treated as binary features in a linear regression. for example if the a house has a fireplace with value more than 9. the coeeficient will be added to sales price. 